## 2WikiMultihopQA and Fusion-in-Decoder

In [1]:
import pyterrier as pt
import pyterrier_rag

ModuleNotFoundError: No module named 'pyterrier_rag'

## Fusion in Decoder (FiD)

FiD is a reader model that takes passages, encodes them separately with the question (e.g. using T5), and then concatenates the representations for the decoder to generate the answer.

In [ ]:
import pyterrier_rag.readers
# fid = pyterrier_rag.readers.T5FiD("/Users/craig.macdonald/Downloads/converted_nq_reader_base", tokenizer_name_or_path='t5-base')
fid = pyterrier_rag.readers.T5FiD("terrierteam/t5fid_base_2wiki")

/opt/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## 2WikiMultihopQA Dataset

We're doing experiments with 2WikiMultihopQA. This dataset comes with passages for each question already, so `dataset.get_topics()` provides their title and text.

In [ ]:
dataset = pt.get_dataset('rag:2wikimultihopqa')
test_answers = dataset.get_answers('dev')
test_docs = dataset.get_topics('dev')
test_docs.head(2)

Reading 2WikiMultihopQA test.json: 100%|██████████| 167454/167454 [00:44<00:00, 3787.25it/s]


,qid,query,docno,title,text
0,00014cad096511ebbdafac1f6bf848b6,"Which one was established first, Hbos or Black...",00014cad096511ebbdafac1f6bf848b6_00_00,NMC Jhapa,The Nepal Multipurpose Cooperative Society Ltd...
1,00014cad096511ebbdafac1f6bf848b6,"Which one was established first, Hbos or Black...",00014cad096511ebbdafac1f6bf848b6_00_01,NMC Jhapa,It was established first with Shree Bachat Tat...


In [ ]:
print("Average nbr of passages per query:", test_docs.groupby('qid').count().mean()['text'])
print("Average passage length (chars):", test_docs['text'].str.len().mean())

Average nbr of passages per query: 30.918938932482952
Average passage length (chars): 103.20824336069532


Lets cutdown the dataset a little in order to speed up experiments - say only 100 question with gold answers.

In [ ]:
test_answers = test_answers.head(100)
test_docs = test_docs.merge(test_answers[['qid']])

# pt.Experiment

Finally, lets evaluate FiD in terms of F1 and EM%. We provide pt.Experiment with 
1. The system(s) to evaluate
2. The input to FiD - i.e. the questions and pasages dataframe
3. The gold answers dataframe
4. The measures we'd like to calculate

In [ ]:
pt.Experiment(
    [fid],
    test_docs,
    test_answers,
    [pyterrier_rag.measures.F1, pyterrier_rag.measures.EM], 
    names=['fid'], verbose=True
)

pt.Experiment: 100%|██████████| 1/1 [04:30<00:00, 270.10s/system]


,name,F1,EM%
0,fid,0.075667,0.05
